In [10]:
import pandas as pd
import numpy as np


In [11]:
import pandas as pd
from openai import OpenAI
import json
import time
from tqdm import tqdm
import os

In [2]:
# Read the CSV file
df = pd.read_csv(r"C:\Users\Laetitia\Documents\Python\hotels_th\hotel_data_20241002_091611.csv")
pd.set_option('display.max_columns', 1000, 'display.width', 1000, 'display.max_rows',1000)
df.sample(2)


,Unnamed: 0,Date Scraped,Hotel Name,Second Span Text,Link URL
365,365,2024-10-02,"D VAREE JOMTIEN BEACH, PATTAYA",NaN,http://www.djb.dvaree.com/
681,681,2024-10-02,SAII PHI PHI ISLAND VILLAGE,Krabi,http://www.saiiresorts.com/


In [3]:
# Clean the 'Hotel Name' column:
# 1. Remove quotes
# 2. Strip leading and trailing whitespace
# 3. Replace multiple spaces with single space
df['Hotel Name'] = df['Hotel Name'].apply(lambda x: str(x)
                                         .replace('"', '')  # Remove quotes
                                         .strip()           # Remove leading/trailing whitespace
                                         .replace('  ', ' '))  # Replace double spaces with single space


In [4]:
df.sample(10)

,Unnamed: 0,Date Scraped,Hotel Name,Second Span Text,Link URL
728,728,2024-10-02,BANYAN TREE PHUKET,Phuket,https://www.banyantree.com/
714,714,2024-10-02,ANDAMAN EMBRACE PATONG,Phuket,http://www.andamanembrace.com/
1019,1019,2024-10-02,"MITUTOYO (THAILAND) CO.,LTD.",Bangkok,http://www.mitutoya.co.th/
159,159,2024-10-02,MANDARIN HOTEL,Bangkok,http://www.mandarin-bkk.com/
32,32,2024-10-02,BANGKOK HOTEL LOTUS SUKHUMVIT,Bangkok,http://www.hotellotussukhumvit.com/
193,193,2024-10-02,OAKWOOD STUDIOS SUKHUMVIT BANGKOK,Bangkok,http://www.oakwood.com/thailand
146,146,2024-10-02,LANDMARK BANGKOK HOTEL,Bangkok,http://www.landmarkbangkok.com/
501,501,2024-10-02,THIPURAI CITY HOTEL,Prachuapkhirikhan,http://www.thipuraicityhotel.com/
184,184,2024-10-02,NORTHGATE RATCHAYOTHIN,Bangkok,http://www.northgatebangkok.com/
24,24,2024-10-02,ASPIRA SKYY,Bangkok,http://www.skyyhotel.com/


In [5]:
df.to_csv('hotel_data_20241002_091611.csv', index=False)

In [7]:
# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] ='sk-9uaMnkNNnNFMrDIhz6gdwDsV8uDNmezZ5EsAx4ycglT3BlbkFJbRiwaqHuwVUnaVc_H3StMmQyXEDq7Z8I-X2Cdj8KQA'


In [8]:
def get_hotel_info_from_openai(hotel_name, hotel_url):
    # Initialize the client
    client = OpenAI()  # Make sure OPENAI_API_KEY is set in your environment variables
    
    try:
        info_prompt = f"""Based on the hotel name "{hotel_name}" and URL {hotel_url}, please provide:
        1. The likely ownership (company or group that owns this hotel) and its name, maybe also include percentage.
        2. If the property is managed by a group, and if so which one.
        3. A possible mission statement for this hotel, keep it short and casual, make it distinctive from any other hotel.
        4. The star rating of this hotel as an integer between 1 and 5.
        
        Respond with a JSON object using this exact format:
        {{
            "ownership": "string describing ownership details",
            "management": "string describing management group, or null if none",
            "mission": "string containing mission statement",
            "star_rating": integer between 1 and 5
        }}"""
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a hotel industry expert analyst. Always provide responses in valid JSON format."},
                {"role": "user", "content": info_prompt}
            ],
            temperature=0.7
        )
        
        # Extract the content and parse JSON
        content = response.choices[0].message.content
        try:
            parsed_content = json.loads(content)
            
            # Ensure star_rating is an integer
            if 'star_rating' in parsed_content:
                parsed_content['star_rating'] = int(parsed_content['star_rating'])
            
            return parsed_content
            
        except json.JSONDecodeError:
            print(f"Failed to parse JSON for {hotel_name}. Raw response: {content}")
            return None
        
    except Exception as e:
        print(f"Error processing {hotel_name}: {str(e)}")
        return None

In [9]:
def get_hotel_info_from_openai(hotel_name, hotel_url):
    try:
        info_prompt = f"""Based on the hotel name "{hotel_name}" and URL {hotel_url}, please provide:
        1. The likely ownership (company or group that owns this hotel) and its name, maybe also include percentage.
        2. If the property is managed by a group, and if so which one.
        3. A possible mission statement for this hotel, keep it short and casual, make it distinctive from any other hotel.
        4. Tell me what is the star rating of this hotel ; The only possible ratings are an integer between 1 and 5."
        
        Format the response as JSON with keys: ownership, management, mission, , star_rating"""
        
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a hotel industry expert analyst."},
                {"role": "user", "content": info_prompt}
            ],
            temperature=0.7
        )
        
        content = response.choices[0].message.content
        
        # Here you might need to add proper JSON parsing depending on the actual response format
        # For now, we'll use a simple dictionary
        return {
            'ownership': content.get('ownership', None),
            'management': content.get('management', None),
            'mission': content.get('mission', None),
            'star_rating': content.get('star_rating', None),
        }
    except Exception as e:
        print(f"Error processing {hotel_name}: {str(e)}")
        return None


In [30]:
def process_hotel_data(csv_path):
    # Read the CSV file
    df = pd.read_csv(csv_path)
    
    # Add or ensure columns exist
    new_columns = ['Ownership', 'Management', 'Mission', 'Star_Rating']
    for col in new_columns:
        if col not in df.columns:
            df[col] = None
    
    # Process each hotel
    for index, row in tqdm(df.iterrows(), total=len(df)):
        if pd.isna(row['Ownership']) or pd.isna(row['Star_Rating']):
            hotel_info = get_hotel_info_from_openai(row['Hotel Name'], row['Link URL'])
            
            if hotel_info:
                for key in hotel_info:
                    column_name = key.title() if key != 'star_rating' else 'Star_Rating'
                    df.at[index, column_name] = hotel_info[key]
            
            # Save progress every 5 hotels
            if (index + 1) % 5 == 0:
                progress_path = csv_path.replace('.csv', f'_enriched_progress_{index+1}.csv')
                df.to_csv(progress_path, index=False)
                print(f"Progress saved: {index+1} hotels processed")
            
            time.sleep(1)  # Rate limiting
    
    # Save final results
    output_path = csv_path.replace('.csv', '_enriched_final.csv')
    df.to_csv(output_path, index=False)
    return df


In [31]:
input_csv_path = "hotel_data_20241002_091611.csv"
enriched_df = process_hotel_data(input_csv_path)

  0%|          | 4/1078 [00:11<52:23,  2.93s/it]

Progress saved: 5 hotels processed


  1%|          | 9/1078 [00:24<44:34,  2.50s/it]

Progress saved: 10 hotels processed


  1%|▏         | 14/1078 [00:38<47:40,  2.69s/it]

...

 99%|█████████▉| 1069/1078 [1:05:52<00:23,  2.66s/it]

Progress saved: 1070 hotels processed


100%|█████████▉| 1074/1078 [1:06:05<00:10,  2.69s/it]

Progress saved: 1075 hotels processed


100%|██████████| 1078/1078 [1:06:16<00:00,  3.69s/it]


In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from pyvis.network import Network
import numpy as np

In [33]:
def create_hotel_network_visualization(csv_path):
    # Read the CSV file
    df = pd.read_csv(csv_path)
    
    # Filter for 5-star hotels
    five_star_hotels = df[df['Star_Rating'] == 5].copy()
    
    # Function to extract main company name
    def extract_main_company(text):
        if pd.isna(text):
            return "Unknown"
        # You might need to adjust this based on your actual data
        return text.split()[0].strip('.,')
    
    # Extract main company names
    five_star_hotels['Main_Owner'] = five_star_hotels['Ownership'].apply(extract_main_company)
    five_star_hotels['Main_Manager'] = five_star_hotels['Management'].apply(extract_main_company)
    
    # Create a NetworkX graph
    G = nx.Graph()
    
    # Add nodes and edges
    for _, hotel in five_star_hotels.iterrows():
        hotel_name = hotel['Hotel Name']
        owner = hotel['Main_Owner']
        manager = hotel['Main_Manager']
        
        G.add_node(hotel_name, type='hotel')
        G.add_node(owner, type='owner')
        G.add_node(manager, type='manager')
        
        G.add_edge(hotel_name, owner)
        if manager != "Unknown":
            G.add_edge(hotel_name, manager)
    
    # Create interactive visualization using pyvis
    nt = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
    
    # Add nodes with different colors for hotels, owners, and managers
    for node in G.nodes():
        node_type = G.nodes[node]['type']
        if node_type == 'hotel':
            nt.add_node(node, label=node, color="#1f78b4")
        elif node_type == 'owner':
            nt.add_node(node, label=node, color="#33a02c")
        else:  # manager
            nt.add_node(node, label=node, color="#e31a1c")
    
    # Add edges
    for edge in G.edges():
        nt.add_edge(edge[0], edge[1])
    
    # Generate the interactive visualization
    nt.show("hotel_network.html")
    
    return five_star_hotels


In [34]:
def create_summary_statistics(five_star_hotels):
    # Create summary DataFrames
    owner_summary = five_star_hotels['Main_Owner'].value_counts().reset_index()
    owner_summary.columns = ['Owner', 'Count']
    
    manager_summary = five_star_hotels['Main_Manager'].value_counts().reset_index()
    manager_summary.columns = ['Manager', 'Count']
    
    return owner_summary, manager_summary

In [35]:
def plot_summaries(owner_summary, manager_summary):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot owner summary
    sns.barplot(data=owner_summary.head(10), x='Count', y='Owner', ax=ax1)
    ax1.set_title('Top 10 Hotel Owners')
    
    # Plot manager summary
    sns.barplot(data=manager_summary.head(10), x='Count', y='Manager', ax=ax2)
    ax2.set_title('Top 10 Hotel Managers')
    
    plt.tight_layout()
    plt.savefig('hotel_summaries.png')
    plt.close()

In [73]:
input_csv_path = "hotel_data_20241002_091611_enriched_final.csv"
df1 = pd.read_csv(input_csv_path)   
df1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1078 entries, 0 to 1077
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        1078 non-null   int64  
 1   Date Scraped      1078 non-null   object 
 2   Hotel Name        1078 non-null   object 
 3   Second Span Text  840 non-null    object 
 4   Link URL          1078 non-null   object 
 5   Ownership         1073 non-null   object 
 6   Management        968 non-null    object 
 7   Mission           1073 non-null   object 
 8   Star_Rating       1073 non-null   float64
dtypes: float64(1), int64(1), object(7)
memory usage: 75.9+ KB


In [74]:
import re
import pandas as pd
import numpy as np

def clean_ownership(ownership_str):
    # Check if the value is a string
    if isinstance(ownership_str, str):
        # Case 1: Company name appears before "owns"
        match_owns = re.search(r'^(.*?)(?:\s+owns|\s+owns\s+\d+%)', ownership_str, re.IGNORECASE)
        
        if match_owns:
            return match_owns.group(1).strip()

        # Case 2: Pattern "is owned by" with company name after it
        match_owned_by = re.search(r'is owned by\s+(.*?)(?:\s+with|$)', ownership_str, re.IGNORECASE)
        
        if match_owned_by:
            return match_owned_by.group(1).strip()

        # Case 3: Extract before comma, e.g. "Maduzi Hotel Group, 100%"
        match_comma = re.search(r'^(.*?)(?:,|$)', ownership_str)
        
        if match_comma:
            return match_comma.group(1).strip()
    
    # If it's not a string or no match is found, return np.nan
    return np.nan

In [75]:
# Apply the function to the column
df1['clean_ownership'] = df1['Ownership'].apply(clean_ownership)


In [76]:
df1.sample()

,Unnamed: 0,Date Scraped,Hotel Name,Second Span Text,Link URL,Ownership,Management,Mission,Star_Rating,clean_ownership
34,34,2024-10-02,BANGKOK PALACE HOTEL,Bangkok,https://www.bangkokpalace.com/,Bangkok Hospitality Group owns 70% of the hotel,The property is self-managed,"Where tradition meets luxury, creating unforge...",4.0,Bangkok Hospitality Group


In [82]:
df1.to_csv("hotel_data_20241002_final_owner.csv", index=False)

In [79]:
import re
import pandas as pd
import numpy as np

def extract_keyword(ownership_str):
    # Ensure we only process strings
    if isinstance(ownership_str, str):
        # Dictionary of patterns to match and their corresponding keywords
        patterns = {
            r'marriott': 'Marriott',
            r'accor': 'Accor',
            r'minor': 'Minor',
            r'intercontinental|ihg': 'InterContinental',
            r'dusit': 'Dusit',
            r'centara': 'Centara',
            r'hilton': 'Hilton',
            r'banyan tree': 'Banyan Tree',
            r'hyatt': 'Hyatt',
            r'ascott': 'Ascott',
            r'mövenpick|moevenpick': 'Mövenpick',
            r'oakwood': 'Oakwood',
            r'imperial': 'Imperial',
            r'pinnacle': 'Pinnacle',
            r'furama': 'Furama',
            r'royal hospitality': 'Royal Hospitality',
            r'santhiya': 'Santhiya',
            r'bandara': 'Bandara',
            r'heritage': 'Heritage',
            r'centre point': 'Centre Point'
        }

        # Iterate over patterns and return the corresponding keyword if a match is found
        for pattern, keyword in patterns.items():
            if re.search(pattern, ownership_str, re.IGNORECASE):
                return keyword

    # If it's not a string or no match is found, return np.nan
    return ownership_str



# Apply the function to the column
df1['keyword_ownership'] = df1['Ownership'].apply(extract_keyword)



                                        clean_ownership                                  keyword_ownership
0     137 Pillars Hotels and Resorts (Thailand) Co.,...  137 Pillars Hotels and Resorts (Thailand) Co.,...
1                                          7 Days Group                                       7 Days Group
2                                         Adelphi Group                                      Adelphi Group
3                   AETAS Hotels & Residences Co., Ltd.                AETAS Hotels & Residences Co., Ltd.
4                               Akara Hospitality Group                            Akara Hospitality Group
...                                                 ...                                                ...
1073                                       FLORAL Group                                       FLORAL Group
1074                         Shanghai Hospitality Group                         Shanghai Hospitality Group
1075                             Marr

In [23]:
df = pd.read_csv('hotel_data_20241002_final_owner.csv')
df.sample(2)

,Hotel Name,Second Span Text,Link URL,Ownership,Management,Mission,Star_Rating,clean_ownership,keyword_ownership
8,AMANTA HOTEL & RESIDENCE SATHORN,Bangkok,https://visitamanta.com/th/sathorn/,Amanta Group owns 100% of the hotel,Managed independently,Where modern luxury meets warm hospitality,4.0,Amanta Group,Amanta Group
102,THE HERITAGE @ SRINAKARIN,Bangkok,http://www.theheritagesrinakarin.com/,"The Heritage Group, 100%",Independent,Where tradition meets modern comfort for a uni...,4.0,The Heritage Group,Heritage


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1099 entries, 0 to 1098
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Hotel Name         1099 non-null   object 
 1   Second Span Text   840 non-null    object 
 2   Link URL           1078 non-null   object 
 3   Ownership          1094 non-null   object 
 4   Management         968 non-null    object 
 5   Mission            1073 non-null   object 
 6   Star_Rating        1073 non-null   float64
 7   clean_ownership    1094 non-null   object 
 8   keyword_ownership  1094 non-null   object 
dtypes: float64(1), object(8)
memory usage: 77.4+ KB


In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Step 1: Fetch the webpage content
url = 'https://www.assetworldcorp-th.com/en/portfolio/hospitalities'
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Step 2: Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Step 3: Find all <h2> elements with the specified class
    hotel_names = soup.find_all('h2', class_='card__title text-uppercase')
    
    # Step 4: Extract text and clean it
    hotel_names_list = [hotel.get_text(strip=True) for hotel in hotel_names]
    
    # Step 5: Load the existing CSV file
    df = pd.read_csv('hotel_data_20241002_final_owner.csv')
    
    # Step 6: Create a DataFrame for the new entries
    new_entries = pd.DataFrame({
        'Hotel Name': hotel_names_list,
        'Second Span Text': [None] * len(hotel_names_list),  # Fill with None or appropriate data
        'Link URL': [url] * len(hotel_names_list),           # Set Link URL to the base URL
        'Ownership': ['Asset World Corp'] * len(hotel_names_list),
        'Management': [None] * len(hotel_names_list),       # Fill with None or appropriate data
        'Mission': [None] * len(hotel_names_list),          # Fill with None or appropriate data
        'Star_Rating': [5] * len(hotel_names_list),         # Set Star Rating to 5
        'clean_ownership': ['Asset World Corp'] * len(hotel_names_list),  # Same as Ownership
        'keyword_ownership': ['Asset World Corp'] * len(hotel_names_list)  # Same as Ownership
    })
    
    # Step 7: Append new entries to the existing DataFrame
    df = pd.concat([df, new_entries], ignore_index=True)
    
    print("Hotel names have been appended successfully.")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


Hotel names have been appended successfully.


In [ ]:
# Step 8: Save the updated DataFrame back to the CSV
df.to_csv('hotel_data_20241002_final_owner.csv', index=False)


In [24]:
# Step 8: Check for duplicates in the 'Hotel Name' column
duplicate_hotels = df[df.duplicated(['Hotel Name'], keep=False)]
duplicate_hotels


,Hotel Name,Second Span Text,Link URL,Ownership,Management,Mission,Star_Rating,clean_ownership,keyword_ownership
